Make initializations

In [1]:
import cbh_data_definitions
import pathlib
import os

import optuna
import pytorch_lightning as pl
import mlflow
from ray import tune
import ray
import ray.tune
import ray.tune.search
import ray.tune.search.optuna
from ray.tune.search.optuna import OptunaSearch
from ray.tune.integration.mlflow import mlflow_mixin
from ray.tune.search import ConcurrencyLimiter
from pytorch_lightning.callbacks import (
    RichProgressBar,
)
from ray.tune.integration.pytorch_lightning import TuneReportCallback
import numpy as np
import datetime
import cbh_torch_MLP

In [2]:
# intialize some settings: mlflow, data directory, resources
root_data_directory = pathlib.Path(os.environ["SCRATCH"]) / "cbh_data"

dev_data_path = root_data_directory / "analysis_ready" / "dev_randomized.zarr"
training_data_path = root_data_directory / "analysis_ready" / "train_randomized.zarr"

mlflow_command_line_run = """
    mlflow server --port 5001 --backend-store-uri sqlite:///mlflowSQLserver.db  --default-artifact-root ./mlflow_artifacts/
"""
mlflow_server_address = 'vld425'
mlflow_server_port = 5001
mlflow_server_uri = f'http://{mlflow_server_address}:{mlflow_server_port:d}'
mlflow_artifact_root = pathlib.Path('./mlflow_artifacts/')

hparams_for_mlflow = {}

redefine data

In [3]:
# init data
(
    train_input,
    train_labels,
    _,
) = cbh_data_definitions.load_data_from_zarr(training_data_path)

(
    dev_input, 
    dev_labels, 
    _
) = cbh_data_definitions.load_data_from_zarr(dev_data_path)

# the cloud volume is not needed for the task, so isn't saved on the load
# show a chunk
train_input

Loaded zarr, file information:
 Name              : /
Type              : zarr.hierarchy.Group
Read-only         : False
Synchronizer type : zarr.sync.ThreadSynchronizer
Store type        : zarr.storage.DirectoryStore
No. members       : 2
No. arrays        : 2
No. groups        : 0
Arrays            : cloud_base_label_y.zarr, humidity_temp_pressure_x.zarr
 

Loaded zarr, file information:
 Name              : /
Type              : zarr.hierarchy.Group
Read-only         : False
Synchronizer type : zarr.sync.ThreadSynchronizer
Store type        : zarr.storage.DirectoryStore
No. members       : 2
No. arrays        : 2
No. groups        : 0
Arrays            : cloud_base_label_y.zarr, humidity_temp_pressure_x.zarr
 



dask.array<from-zarr, shape=(111820800, 70, 3), dtype=float32, chunksize=(2329600, 70, 3), chunktype=numpy.ndarray>

In [4]:
# limit the data by a factor for less data in a tuning trial
factors_of_chunk = [n for n in range(1, train_input.chunksize[0] + 1) if train_input.chunksize[0] % n == 0]
print("Factors of chunk: ", factors_of_chunk)
hparams_for_mlflow['Limited sample number'] =  -1

Factors of chunk:  [1, 2, 4, 5, 7, 8, 10, 13, 14, 16, 20, 25, 26, 28, 32, 35, 40, 50, 52, 56, 64, 65, 70, 80, 91, 100, 104, 112, 128, 130, 140, 160, 175, 182, 200, 208, 224, 256, 260, 280, 320, 325, 350, 364, 400, 416, 448, 455, 512, 520, 560, 640, 650, 700, 728, 800, 832, 896, 910, 1024, 1040, 1120, 1280, 1300, 1400, 1456, 1600, 1664, 1792, 1820, 2080, 2240, 2275, 2560, 2600, 2800, 2912, 3200, 3328, 3584, 3640, 4160, 4480, 4550, 5120, 5200, 5600, 5824, 6400, 6656, 7168, 7280, 8320, 8960, 9100, 10400, 11200, 11648, 12800, 13312, 14560, 16640, 17920, 18200, 20800, 22400, 23296, 25600, 29120, 33280, 35840, 36400, 41600, 44800, 46592, 58240, 66560, 72800, 83200, 89600, 93184, 116480, 145600, 166400, 179200, 232960, 291200, 332800, 465920, 582400, 1164800, 2329600]


setup study

In [5]:
# DEFINE ALL SETTINGS FOR TRAINING, includes hparam space
experiment_name = 'cbh-hparam-tuning'
CPU_COUNT = 10
RAM_GB = 128
hparams_for_mlflow['CPU Count'] = CPU_COUNT
hparams_for_mlflow['Compute Memory'] = RAM_GB
thread_count_for_dask = CPU_COUNT
dataset_method = '1chunk'
randomize_chunkwise_1chunk = False
shuffle_train_data = False
collate_fn = None # alt: cbh_data_definitions.dataloader_collate_with_dask
num_workers_dataloader = 0 # alt: CPU_COUNT +-
global_trail_number = 0
max_time_for_trial = "00:02:00:00"  # dd:hh:mm:ss
hparams_for_mlflow["Training timeout"] = max_time_for_trial

max_node_num_exclusive = 513
factors_for_hparam_choice = [factor for factor in factors_of_chunk if (factor<3300 and factor>3)]
mlp_search_space = {
    "epoch": 1,
    "lr": tune.quniform(0.0001, 0.01, 0.00005),
    "data_limit": 4,#tune.randint(4, int(len(train_labels.chunks[0]) / 4)), # multiple chunk ind by chunklen: train_input.chunksize[0])
    # "activation": tune.choice(["relu", "tanh"]),
    "batch_size": tune.choice(factors_for_hparam_choice),
    "arch_name":"MLP",
    "hidden_layers":tune.randint(1,11),
    "activation":tune.choice(["relu", "tanh"]),
    "input_size":(train_input.shape[2] * train_input.shape[1]),
    "output_size": train_input.shape[1],
    "layer_node_num": tune.sample_from(lambda spec: 8*np.random.randint(1,int(max_node_num_exclusive/8), size=spec.config.hidden_layers)),
    "deterministic":True,
    "chkpt_time":datetime.timedelta(minutes=15),
    "max_time":max_time_for_trial
    
}

In [6]:
class MLFlowLogger(pl.loggers.MLFlowLogger): #overwrite mlflogger
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    def after_save_checkpoint(self, model_checkpoint: pl.callbacks.ModelCheckpoint) -> None:
        """
        Called after model checkpoint callback saves a new checkpoint.
        """
        best_chkpt = torch.load(model_checkpoint.best_model_path)
        checkpoint_for_mlflow = {
            "val loss": float(best_chkpt['callbacks'][list(key for key in list(best_chkpt['callbacks'].keys()) if "ModelCheckpoint" in key)[0]]['current_score']),
            "train loss at step-1": list(train_loss_metric.value for train_loss_metric in mlf_logger._mlflow_client.get_metric_history(run.info.run_id, "Train loss") if (int(train_loss_metric.step) == int(best_chkpt['global_step']-1)))[0],
            "global_step": best_chkpt['global_step'],
            "model_state_dict": best_chkpt['state_dict'],
            "checkpoint": best_chkpt,
        }
        with TemporaryDirectory() as tmpdirname:
            f_name = os.path.join(tmpdirname, f"{run.info.run_id}-best_model_checkpoint-step_{best_chkpt['global_step']}.pt")
            torch.save(checkpoint_for_mlflow, f_name)
            mlflow.log_artifact(f_name)

mlflow.set_tracking_uri(mlflow_server_uri)
# make vars global
mlf_exp = None
mlf_exp_id = None
try: 
    print('Creating experiment')
    mlf_exp_id = mlflow.create_experiment(experiment_name)
    mlf_exp = mlflow.get_experiment(mlf_exp_id)
except mlflow.exceptions.RestException as e:
    print("Caught")
    if False:
        print(e)
    mlf_exp = mlflow.get_experiment_by_name(experiment_name)
print("Success")

mlp_search_space["mlflow"] = {
    "tracking_uri":mlflow_server_uri,
    "experiment_id":mlf_exp_id,
    "experiment_name":experiment_name,
    # "run_name":(experiment_name+str(datetime.datetime.now())),
}

Creating experiment
Caught
Success


In [7]:
@mlflow_mixin
def objective(ray_config):
    # def model hparams with config
    # print(ray_config)
    # print(dir(ray_config))
    # def data
    print("Test print")
    datamodule = cbh_data_definitions.CBH_DataModule(
        train_input, train_labels,
        dev_input, dev_labels,
        thread_count_for_dask,
        ray_config['batch_size'],
        num_workers = num_workers_dataloader,
        collate_fn = collate_fn,
        shuffle = shuffle_train_data,
        randomize_chunkwise = randomize_chunkwise_1chunk,
        method=dataset_method,
    )
    #def model
    model = cbh_torch_MLP.CloudBaseMLP(
        ray_config['input_size'],
        ray_config['layer_node_num'],
        ray_config['output_size'],
        ray_config['hidden_layers'],
        ray_config['activation'],
        ray_config['lr'],
    )
    # def experiment naming
    timestamp_template = '{dt.year:04d}{dt.month:02d}{dt.day:02d}T{dt.hour:02d}{dt.minute:02d}{dt.second:02d}'
    run_name_template = 'cbh_challenge_{network_name}_' + timestamp_template
    global global_trail_number
    current_run_name = run_name_template.format(network_name=model.__class__.__name__,
                                                    dt=datetime.datetime.now()
                                                   )
    print("Finished model init")
    # begin mlflow experiment run
    with mlflow.start_run(experiment_id=mlf_exp.experiment_id, run_name=current_run_name, nested=True) as run:
        print("Started mlflow run")
        mlflow.pytorch.autolog()
        mlf_logger = MLFlowLogger(experiment_name=experiment_name, tracking_uri=mlflow_server_uri, run_id=run.info.run_id)
        print("Finished init logger")
        # define trainer
        time_for_checkpoint = ray_config['chkpt_time']
        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            train_time_interval=time_for_checkpoint,
            dirpath=run.info.artifact_uri,
            monitor="val_loss_mean",
            save_on_train_epoch_end=False,
            mode="min"
        )
        callbacks = [checkpoint_callback, RichProgressBar(), TuneReportCallback({'val_loss_mean': 'val_loss_mean'}, on="validation_end")]
        print("Finished define callbacks")
        trainer_hparams = {
            'max_epochs':ray_config['epoch'],
            'deterministic':ray_config['deterministic'],
            'val_check_interval':10, # val every percentage of the epoch or an INT for after a number of batches
            # 'devices':"auto",
            # 'accelerator':"auto",
            'max_time':ray_config['max_time'],
            'replace_sampler_ddp':False,
            'enable_checkpointing':True,
            'strategy':None,
            'callbacks':callbacks,
            'logger':mlf_logger,
        }
        print("Finished init hparams kwargs")

        hparams_for_mlflow["Trainer hparams"] = trainer_hparams
        # hparams_for_mlflow["Trainer hparams"]['callbacks'] = None # needs improvement, do not log function references, but parameters
        mlf_logger.log_hyperparams(hparams_for_mlflow)
        print("Finished log hparams mlflow")
        trainer = pl.Trainer(
            **trainer_hparams
        )
        print("REACH all init before fit")
        trainer.fit(model=model, datamodule=datamodule)
        path_to_save = '{dt.year:04d}{dt.month:02d}{dt.day:02d}-{dt.hour:02d}{dt.minute:02d}{dt.second:02d}'.format(dt=datetime.datetime.now())
        trainer.save_checkpoint(filepath=run.info.artifact_uri + f'/post_epoch_modelchkpt_{path_to_save}')

In [ ]:
searcher = OptunaSearch(metric=["val_loss_mean"], mode=["min"])
algo = ConcurrencyLimiter(searcher, max_concurrent=int(CPU_COUNT*(3/4)))
num_hparam_trials = 1

tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        # search_alg=algo,
        num_samples=num_hparam_trials,
    ),
    param_space=mlp_search_space,
)
results = tuner.fit()

2022-11-14 17:01:07,017	INFO worker.py:1518 -- Started a local Ray instance.
2022-11-14 17:01:13,799	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,activation,batch_size,hidden_layers,layer_node_num,lr,iter,total time (s),val_loss_mean
objective_f23af_00000,RUNNING,10.154.1.80:32965,relu,1300,4,[440 480 208 456],0.005,6,110.479,3.1625


(objective pid=32965) Test print
(objective pid=32965) Finished model init
(objective pid=32965) Started mlflow run
(objective pid=32965) Finished init logger
(objective pid=32965) Finished define callbacks
(objective pid=32965) Finished init hparams kwargs


(objective pid=32965) /home/h02/hsouth/.conda/envs/py-lightning/lib/python3.10/site-packages/pytorch_lightning/loggers/mlflow.py:225: RuntimeWarning: Mlflow only allows parameters with up to 250 characters. Discard Trainer hparams/callbacks=[<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x2b0a101e7850>, <pytorch_lightning.callbacks.progress.rich_progress.RichProgressBar object at 0x2b0a102255a0>, <ray.tune.integration.pytorch_lightning.TuneReportCallback object at 0x2b0a10225600>]
(objective pid=32965)   rank_zero_warn(
(objective pid=32965) GPU available: False, used: False
(objective pid=32965) TPU available: False, using: 0 TPU cores
(objective pid=32965) IPU available: False, using: 0 IPUs
(objective pid=32965) HPU available: False, using: 0 HPUs
(objective pid=32965) /home/h02/hsouth/.conda/envs/py-lightning/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:229: LightningDeprecationWarning: The `on_init_start` callback hook

(objective pid=32965) Finished log hparams mlflow
(objective pid=32965) REACH all init before fit
(objective pid=32965) ┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
(objective pid=32965) ┃   ┃ Name              ┃ Type             ┃ Params ┃
(objective pid=32965) ┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
(objective pid=32965) │ 0 │ layer_norm        │ LayerNorm        │    420 │
(objective pid=32965) │ 1 │ linears           │ ModuleList       │  531 K │
(objective pid=32965) │ 2 │ normalize_outputs │ Softmax          │      0 │
(objective pid=32965) │ 3 │ crossentropy_loss │ CrossEntropyLoss │      0 │
(objective pid=32965) └───┴───────────────────┴──────────────────┴────────┘
(objective pid=32965) Trainable params: 532 K                                                         
(objective pid=32965) Non-trainable params: 0                                                         
(objective pid=32965) Total params: 532 K                                               

(objective pid=32965) /home/h02/hsouth/.conda/envs/py-lightning/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(objective pid=32965)   rank_zero_warn(
(objective pid=32965) /home/h02/hsouth/.conda/envs/py-lightning/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(objective pid=32965)   rank_zero_warn(


Result for objective_f23af_00000:
  date: 2022-11-14_17-01-44
  done: false
  experiment_id: a80334bd00584e7281528d4c609f96c6
  hostname: expspicesrv066
  iterations_since_restore: 1
  node_ip: 10.154.1.80
  pid: 32965
  time_since_restore: 24.6245174407959
  time_this_iter_s: 24.6245174407959
  time_total_s: 24.6245174407959
  timestamp: 1668445304
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: f23af_00000
  val_loss_mean: 3.562089443206787
  warmup_time: 0.1783595085144043
  
Result for objective_f23af_00000:
  date: 2022-11-14_17-02-00
  done: false
  experiment_id: a80334bd00584e7281528d4c609f96c6
  hostname: expspicesrv066
  iterations_since_restore: 2
  node_ip: 10.154.1.80
  pid: 32965
  time_since_restore: 40.811378717422485
  time_this_iter_s: 16.186861276626587
  time_total_s: 40.811378717422485
  timestamp: 1668445320
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: f23af_00000
  val_loss_mean: 3.2160582542419434
  warmup_time: 0.17835950851

2022-11-14 17:03:21,116	WARNING tune.py:686 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


ensure mlflow

run study

eval